# Home Assignment 2 (30 points)

Submit your solutions in teams of 5 students. Unless explicitly agreed otherwise in advance, **submissions from teams with more or less members will NOT be graded**.
List all members of the team with their student ID in the cell below, and submit only one notebook per team. Only submit a notebook, do not submit the dataset(s) you used. Also, do NOT compress/zip your submission (incorrect submissions get 0 points)!

You may use the code from the exercises and basic functionalities that are explained in official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members may be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points. We use the conda environment given below.
* If you use packages that are not available on the default or conda-forge channel, list them below. Also add a link to installation instructions.
* Demonstrate that your code works by providing outputs for test inputs or with simple unit tests (`assert actual == expected`).
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Avoid reusing variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post on Slack, so we can clear up such questions for all students in the course.


Conda environment used for grading:

```yml
name: nlp-ss2024
channels:
  - defaults
dependencies:
  - python=3.11
  - pip
  - pip:
    - numpy==1.26.4
    - nltk==3.8.1
    - scikit-learn==1.4.2
    - scipy==1.13.0
    - notebook
```


### AI Disclosure

Did you use any AI assistance to complete this homework? If so, please also specify what AI you used.
   - [ ] Yes, we used the following assistants: ....
   - [ ] No 

---
*(only complete the below questions if you answered yes above)*

* If you used a large language model to assist you, please paste *all* of the prompts that you used below. Add a separate bullet for each prompt, and specify which problem is associated with which prompt.
    * *your response here*
* **Free response**: For each problem for which you used assistance, describe your overall experience with the AI. How helpful was it? Did it just directly give you a good answer, or did you have to edit it? Was its output ever obviously wrong or irrelevant? Did you use it to get the answer or check your own answer?
    * *your response here*

In [86]:
import sys

# UMR Usernames of all group members (<Username>@students.uni-marburg.de)
team_members = [
    "Student1",
    "Student2",
    "..."
]

Additional packages (if any):
 - Example: `powerlaw`, https://github.com/jeffalstott/powerlaw

In [87]:
from typing import List, Union, Dict, Set, Tuple, Sequence
from numpy.typing import NDArray

### Task 1: WordNet word similarity

In this task, we want to implement the similarity between two words in WordNet (https://www.nltk.org/api/nltk.corpus.reader.wordnet.html) using the NLTK package. The word similarity between two words is given by
$$
\frac{1}{1+d}
$$
where $d$ is the distance of the shortest path in the hypernym/hyponym hierarchy tree in WordNet between any pair of synsets that are associated with the two input words.

From NLTK you should __only__ use the `synsets`, `hypernyms` and `instance_hpyernyms` functions.

The following subtasks build on each other, i.e. the functions of the preceding subtasks can be used for the current subtask.

_Note: the distance of a synset to itself is 0, the distance to a direct hypernym is 1, ..._

In [88]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset

__a)__ Write a function ``shortest_paths_to`` that takes a synset as input and computes the shortest paths to all nodes on the way to the root in the hypernym hierarchy tree of WordNet. The function should return a dictionary that matches all visited hypernyms on the way(s) to the root to the distance of the shortest path from the input synset. Consider that a synset might have multiple paths to the root and that some nodes might appear in multiple paths. However, we only want to store the shortest distances. Moreover, keep in mind that the input synset might be an instance. 

Use the signature in the cell below.

__Example:__ _Calling_ ``shortest_paths_to(s)`` _on the synset_ ``s = wn.synset('calculator.n.01')`` _should yield the following result:_

``
{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 1,
 Synset('person.n.01'): 2,
 Synset('causal_agent.n.01'): 3,
 Synset('organism.n.01'): 3,
 Synset('physical_entity.n.01'): 4,
 Synset('living_thing.n.01'): 4,
 Synset('entity.n.01'): 5,
 Synset('whole.n.02'): 5,
 Synset('object.n.01'): 6}
``

In [89]:
def shortest_paths_to(start_syn: Synset) -> Dict[Synset, int]:
    """Compute the shortest distance to all nodes on paths to the root.
    :param start_syn: synset to which we want to compute the shortest distances
    :return: dict that matches all visited hypernyms to their distance to the input synset
    """
    paths = {}
    for path in start_syn.hypernym_paths():
        i = len(path)-1
        for node in path:
            if node not in paths:
                paths[node] = i
            else:
                paths[node] = min(paths[node], i)
            i = i-1
    # your code here
    return paths

In [90]:

a = shortest_paths_to(wn.synset('calculator.n.01'))
b = {
    wn.synset('calculator.n.01'): 0,
    wn.synset('expert.n.01'): 1,
    wn.synset('person.n.01'): 2,
    wn.synset('causal_agent.n.01'): 3,
    wn.synset('organism.n.01'): 3,
    wn.synset('physical_entity.n.01'): 4,
    wn.synset('living_thing.n.01'): 4,
    wn.synset('entity.n.01'): 5,
    wn.synset('whole.n.02'): 5,
    wn.synset('object.n.01'): 6
}
assert a == b

__b)__ Write a function ``merge_paths`` that gets two dictionaries that map synsets to shortest distances (you can assume they were created by the function from __a)__) and merges them. The function shold return a dictionary that includes all synsets and distances that appear in any of the input dictionaries. If a synset appears in both input dictionaries, we want to keep the shorter distance. Leave the input dictionaries unaltered.

Use the signature in the cell below.

In [91]:
def merge_paths(p1: Dict[Synset, int], p2: Dict[Synset, int]) -> Dict[Synset, int]:
    """Merge two paths keeping the shorter distance for synsets that appear more than once.
    :param p1: first dict that maps synsets to their shortest distances
    :param p2: second dict that maps synsets to their shortest distances
    :return: merged dict
    """
    merged_paths = p1
    for key in p2:
        if key in merged_paths:
            merged_paths[key] = min(merged_paths[key], p2[key])
        else:
            merged_paths[key] = p2[key]
    # your code here
    return merged_paths

In [92]:
b = shortest_paths_to(wn.synset('phone.n.01'))
merged = merge_paths(a,shortest_paths_to(wn.synset('phone.n.01')))
print(merged)

{Synset('entity.n.01'): 5, Synset('physical_entity.n.01'): 4, Synset('causal_agent.n.01'): 3, Synset('person.n.01'): 2, Synset('expert.n.01'): 1, Synset('calculator.n.01'): 0, Synset('object.n.01'): 6, Synset('whole.n.02'): 5, Synset('living_thing.n.01'): 4, Synset('organism.n.01'): 3, Synset('artifact.n.01'): 4, Synset('instrumentality.n.03'): 3, Synset('equipment.n.01'): 2, Synset('electronic_equipment.n.01'): 1, Synset('telephone.n.01'): 0}


__c)__ Write a function ``all_hypernym_paths`` that gets a word as input and returns a dictionary that maps all hypernyms that are reachable from the set of synsets associated with the word to the shortest distance leading there.

Use the signature in the cell below.

In [93]:
def all_hypernym_paths(word: str) -> Dict[Synset, int]:
    """Get all hypernyms of all synsets associated with the input word and compute the shortest distance leading there.
    :param word: input word
    :return: dict that matches all reachable hypernyms to their shortest distance
    """
    # your code here
    combined = {}
    synsets = wn.synsets(word)
    for synset in synsets:
        shortest_path = shortest_paths_to(synset)
        combined = merge_paths(combined, shortest_path)
    return combined

In [94]:
all_hypernym_paths('calculator')

{Synset('entity.n.01'): 5,
 Synset('physical_entity.n.01'): 4,
 Synset('causal_agent.n.01'): 3,
 Synset('person.n.01'): 2,
 Synset('expert.n.01'): 1,
 Synset('calculator.n.01'): 0,
 Synset('object.n.01'): 6,
 Synset('whole.n.02'): 5,
 Synset('living_thing.n.01'): 4,
 Synset('organism.n.01'): 3,
 Synset('artifact.n.01'): 4,
 Synset('instrumentality.n.03'): 3,
 Synset('device.n.01'): 2,
 Synset('machine.n.01'): 1,
 Synset('calculator.n.02'): 0}

__d)__  Write a function ``get_dist`` that returns the word similarity between two input words, according to the formula given in the task description at the beginning.  

Use the signature in the cell below.

In [108]:
def get_dist(w1 : str, w2 : str) -> float:
    """Compute the similarity between two input words in the WordNet hierarchy tree.
    :param w1: first input word
    :param w2: second input word
    :return: word similarity
    """
    h1 = all_hypernym_paths(w1)
    h2 = all_hypernym_paths(w2)
    
    common_hypernyms = list(set(h1.keys()).intersection(set(h2.keys())))
    d = sys.maxsize
    for common_hypernym in common_hypernyms:
        d1 = h1[common_hypernym]
        d2 = h2[common_hypernym]
        d = min(d, d1+d2) 
    # your code here
    return 1/(d+1)

In [119]:
get_dist('line', 'semicolon')

9


0.1

### Task 2: Lesk algorithm

In this task we want to implement a simple version of the Lesk algorithm, a thesaurus-based method for word sense disambiguation. Given a target word $w$ and a context, the algorithm finds the word sense that is most fitting in the context. To achieve this, the Lesk algorithm computes the number of overlapping words between the context sentence and the definitions of the WordNet synsets, associated with $w$.

Write a function ``lesk`` that takes a word and a context string (e.g. a sentence) and returns the most fitting sense from the synsets associated with the word and the corresponding context overlap. The most fitting sense is the one whose definition shares the most words with the context string. Before matching tokens, make sure to 

* only include valid tokens (cf. HA 1, task 2a)
* remove stopwords
* only match stems of words (e.g. consider the ``PorterStemmer`` from ``nltk``)

When computing the context overlap, count each stemmed word only once, even if they occur multiple times. If there is no fitting synset, i.e. the context overlap between the context and the synset definitions is 0, return None instead.

Use the signature in the cell below.

In [120]:
# HA 1, task 2a)
from nltk.corpus.reader.util import StreamBackedCorpusView
from nltk.corpus import stopwords
import re
import string

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    :param tokens: list of tokens that should be cleaned
    :param remove_stopwords: bool indicating if stopwords should be removed
                             False by default
    :return: list of valid tokens
    """
    valid = []
    punct = string.punctuation
    stop = stopwords.words('english')
    digit = re.compile(r"\d+")

    for t in tokens:
        if t in punct:
            continue
        if remove_stopwords and t.lower() in stop:
            continue
        if re.fullmatch(digit, t):
            continue
        valid.append(t.lower())
    return valid

In [98]:
def lesk(word: str, context: str) -> (Synset, int):
    '''
    Compute the most probable sense of a word in the given context.
    :param word: ambiguous word
    :param context: context in which the word appears
    :returns:
        - synset with the most likely word sense
        - context overlap of synset and context
    '''
    
    
    # your code here
    return

### Task 3: Minimum cost string alignment

In this tak we want to compute an alignment between two strings, that has minimum edit distance. 

Implement a function that takes two strings and their edit distance matrix and returns a minimum cost alignment. You can assume that the edit distance matrix is provided by the function that you implemented in exercise 3, task 2, with a substitution cost of 2. 

A minimum cost alignment consists of two strings that, printed below each other comprise the alignment, where insertions and deletions are represented by a ``*``. Use the function signature in the cell below.

__Example:__ _Given the input strings_ `"INTENTION"` _and_ `"EXECUTION"` _and the corresponding edit distance matrix:_

```
[[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  6  7  8]
 [ 2  3  4  5  6  7  8  7  8  7]
 [ 3  4  5  6  7  8  7  8  9  8]
 [ 4  3  4  5  6  7  8  9 10  9]
 [ 5  4  5  6  7  8  9 10 11 10]
 [ 6  5  6  7  8  9  8  9 10 11]
 [ 7  6  7  8  9 10  9  8  9 10]
 [ 8  7  8  9 10 11 10  9  8  9]
 [ 9  8  9 10 11 12 11 10  9  8]]
```
  
_your function should return the two strings_ ``INTE***NTION`` _and_ ``***EXECUTION`` _that represent the alignment, when printed below each other:_
 
 ``INTE***NTION``    
 ``***EXECUTION`` 
 
 __Remark:__ _The alignment in the example above is not the only solution. In this task all alignments with minimum edit distance are accepted._


In [99]:
def get_alignment(str1: str, str2: str, D: NDArray[NDArray[int]]) -> tuple[str, str]:
    '''
    :param str1: first string for alignment
    :param str2: second string for alignment
    :param D: edit distance matrix of str1 and str2
    :returns: tuple of strings that indicate the alignment of the input strings
    '''
    # your code here
    return